In [16]:
from pinecone import Pinecone
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Pinecone setup
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT", "us-east1-gcp")
PINECONE_INDEX_NAME = "belief-embeddings"

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Given Belief IDs
belief_ids = [
    "13769_belief_15922455", "20358_belief_78401984", "17458_belief_13478500",
    "28169_belief_86425611", "26780_belief_40546367", "25220_belief_63575069",
    "24948_belief_62549133", "227_belief_49613761", "21427_belief_76728810",
    "24244_belief_20219866"
]

for belief_id in belief_ids:
    # Retrieve the belief vector
    fetch_response = index.fetch([belief_id])
    belief_data = fetch_response.vectors.get(belief_id)

    if not belief_data:
        print(f"❌ Belief vector not found for ID: {belief_id}")
        continue

    belief_text = belief_data.metadata.get("belief", "Unknown Belief")
    belief_vector = belief_data.values

    # Query for the top-5 closest concepts
    query_result = index.query(
        vector=belief_vector,
        top_k=20,
        include_metadata=True,
        filter={"type": "concept"}
    )

    # Print belief once
    print(f"\n🔹 **Belief:** {belief_text}")

    # Print top-5 related concepts
    for match in query_result.matches:
        concept_text = match.metadata.get("concept", "Unknown Concept")
        similarity_score = match.score
        print(f"   → {concept_text} (Similarity: {similarity_score:.3f})")



🔹 **Belief:** Rituals are essential for moral and psychological development.
   → Ritual propriety (li 禮) (Similarity: 0.843)
   → Ritual propriety (Similarity: 0.833)
   → Ritual propriety (Similarity: 0.833)
   → Ritual Propriety (Similarity: 0.833)
   → Confucian rituals (Similarity: 0.830)
   → Confucian Social and Moral Order (Similarity: 0.827)
   → Ritual (Similarity: 0.825)
   → Ritual (Similarity: 0.825)
   → Ritual (Similarity: 0.825)
   → Ritual (Similarity: 0.825)
   → Ritual (Similarity: 0.825)
   → Ritual (Similarity: 0.825)
   → Ritual Propriety (Li) (Similarity: 0.825)
   → Ritual propriety (li) (Similarity: 0.825)
   → Rituals (Similarity: 0.823)
   → Rituals (Similarity: 0.823)
   → rituals (Similarity: 0.823)
   → Rituals (Similarity: 0.823)
   → Moral cultivation (Similarity: 0.823)
   → Moral cultivation (Similarity: 0.823)

🔹 **Belief:** Not all predicates express properties.
   → Truth-predicate (Similarity: 0.874)
   → Truth-predicates (Similarity: 0.874)
   → 

Since concepts and beliefs were extracted from documents, there are duplicate Concept nodes, but with separate origins and meanings.
We're going to restructure the nodes to combine synonymous nodes into a new class apart from Belief and Concept.